Don't forget metric beat before!!!

<div class="jumbotron">
    <center>
        <h1>The last mile</h1>
        <h2>From Spark Streaming to Visualization in 10 steps</h2>
    </center>
</div>

![](https://i.pinimg.com/564x/85/88/9a/85889a585974c7331b10343c98a53ee8.jpg)
[Pinterest](https://www.pinterest.ca/pin/255649716331177273/)

 # 1. from spark to elastic search

![](https://static-www.elastic.co/v3/assets/bltefdd0b53724fa2ce/blt7f9ca77d747fb53d/5c240aa34cca137b3874384e/es-hadoop-diagram.svg)

# Doc please
* https://www.elastic.co/what-is/elasticsearch-hadoop
* https://www.elastic.co/guide/en/elasticsearch/hadoop/current/spark.html

![](https://codinginfinite.com/wp-content/uploads/2019/05/documentation-where-were-going-we-dont-need-documentation.jpg)
[Source](https://codinginfinite.com/2019-programming-funny-jokes-memes/)

Ok quite simple first we need a driver to add, we can add to packages to be passed to spark-submit
and maven will do to do the rest

```bash
./sparkSubmitPython.sh twitter_stream_sentiment_es.py "org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.5,org.elasticsearch:elasticsearch-hadoop:jar:7.7.0"
```

# 2. Define the index

Altough Elastic Search do automatically index creation and guess on field, 
sometimes is better to predefine the index and define mapping

This requires add elasticsearch python module to Spark docker image

```Dockerfile 
RUN pip install pyspark numpy elasticsearch
``` 

Then we need to create the mapping

```python
mapping = {
    "mappings": {
        "properties": {
            "timestamp": {
                "type": "date"
            }
        }
    }
}
```

Then connect to Elasticsearch and create the index

```python
elastic = Elasticsearch(hosts=[elastic_host])

# make an API call to the Elasticsearch cluster
# and have it return a response:
response = elastic.indices.create(
    index=elastic_index,
    body=mapping,
    ignore=400 # ignore 400 already exists code
)

if 'acknowledged' in response:
    if response['acknowledged'] == True:
        print ("INDEX MAPPING SUCCESS FOR INDEX:", response['index'])

# catch API error response
elif 'error' in response:
    print ("ERROR:", response['error']['root_cause'])
    print ("TYPE:", response['error']['type']) 
```

![](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTMTXhs9fd7M_qo4XA_6OiKLX9GaTldJWPICDtIGCB7SuE6q9NM&usqp=CAU)

# 3. Define Schema

```python
tweetSchema = tp.StructType([
    tp.StructField(name= 'timestamp', dataType= tp.LongType(),  nullable= True),
    tp.StructField(name= 'tweet',      dataType= tp.StringType(),  nullable= True)
])
```

![](https://www.memecreator.org/static/images/memes/4772258.jpg)

# 4. Configure Elastic Search Parameters

```python
elastic_host="10.0.100.51"
elastic_index="twitter"
elastic_document="_doc"

es_write_conf = {
# specify the node that we are sending data to (this should be the master)
"es.nodes" : elastic_host,
# specify the port in case it is not the default port
"es.port" : '9200',
# specify a resource in the form 'index/doc-type'
"es.resource" : '%s/%s' % (elastic_index,elastic_document),
# is the input JSON?
"es.input.json" : "yes"
}
```

![](http://www.quickmeme.com/img/6d/6d9b3966c39d711a732a842f41bc39f7023eb4fafdc9ecb22d0f8ca1c2fee05e.jpg)
[Source](http://www.quickmeme.com/meme/3tvnt7)

# 5. Extract data from Json

```python
  tweet=rdd.map(lambda (key, value): json.loads(value)).map(
        lambda json_object: (
            json_object["text"].encode('utf-8'), 
            long(json_object["timestamp_ms"])
        )
    )
```

![](https://i.imgflip.com/3ddyn4.jpg)
[Source](https://imgflip.com/i/3ddyn4)

# 6. Convert in RDD Json

```python
new = data.rdd.map(lambda item: {'timestamp': item['timestamp'],'tweet': item['tweet'], 'sentiment':item['prediction']})
final_rdd = new.map(json.dumps).map(lambda x: ('key', x))
```

![](https://i.pinimg.com/originals/fb/fa/78/fbfa7899fc4092a0240021a56a33dc5a.jpg)

# 7. Send to ES

```python
   final_rdd.saveAsNewAPIHadoopFile(
    path='-',
    outputFormatClass="org.elasticsearch.hadoop.mr.EsOutputFormat",
    keyClass="org.apache.hadoop.io.NullWritable",
    valueClass="org.elasticsearch.hadoop.mr.LinkedMapWritable",
    conf=es_write_conf)
 ```

![](https://i.imgflip.com/373hpn.jpg)
[Source](https://imgflip.com/i/373hpn)

# 8. Run

```bash
./sparkSubmitPython.sh twitter_stream_sentiment_es.py "org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.5,org.elasticsearch:elasticsearch-hadoop:jar:7.7.0"
```

![](https://i.chzbgr.com/full/2922317568/h5AC86B91/but-wait-theres-more)

# 9. Define Index Pattern in  Kibana

# 10. start creating visualizations and search

# Biblio

* https://www.elastic.co/what-is/elasticsearch-hadoop
* https://www.elastic.co/guide/en/elasticsearch/hadoop/current/spark.html
* https://medium.com/@akkidx/indexing-into-elasticsearch-using-spark-code-snippets-55eabc753272
* https://github.com/vinayak0792/SparkStreaming/blob/master/consumer.py